In [0]:
# downloading dataset from kaggle
# Если не сработало с первого раза - перезапустить программу

import os

!pip install kaggle --upgrade
!mkdir .kaggle 
import json 
token = {"username":"ephobia","key":"5b1e884343d9f9b18bb2dd73299f3086"} 
try:
  with open('/root/.kaggle/kaggle.json', 'w') as file: 
    json.dump(token, file)
except:
  try:
    with open('.kaggle/kaggle.json', 'w') as file: 
      json.dump(token, file)
  except:
    with open('/root/.kaggle/kaggle.json', 'w') as file: 
      json.dump(token, file)
!KAGGLE_USERNAME="ephobia" 
!KAGGLE_KEY="5b1e884343d9f9b18bb2dd73299f3086" 
!kaggle competitions download -c quickdraw-doodle-recognition -f train_simplified.zip --force

In [0]:
! unzip train_simplified.zip -d train_data
! ls

In [0]:
import numpy as np
import zipfile
import pandas as pd
import tqdm
import matplotlib 
import matplotlib.pyplot as plt
import csv
import os


zf = zipfile.ZipFile("train_simplified.zip")

class_labels = [x.replace(".csv", "") for x in zf.namelist()]


# loop through file eternally
def get_eternal_csv_generator(fn, debug=False):
  while True:
    with open(os.path.join('train_data', fn), "r") as f:
      f.readline()  # skip header
      lines = csv.reader(f, delimiter=',', quotechar='"')
      for line in lines:
        yield line[1], line[5]
      if debug:
        print(fn, "is done, starting from the beginning...")


In [0]:
def raw_batch_generator(batch_size, debug=False):
    generators = np.array([get_eternal_csv_generator(fn, debug) for fn in zf.namelist()])
    while True:
        random_indices = np.random.randint(0, len(generators), size=batch_size)
        yield [next(gen) for gen in generators[random_indices]]


In [7]:
import ast
from PIL import Image, ImageDraw

IMG_SIZE = 224

# faster conversion function
def draw_it(strokes):
    image = Image.new("P", (256, 256), color=255)
    image_draw = ImageDraw.Draw(image)
    for stroke in ast.literal_eval(strokes):
        for i in range(len(stroke[0]) - 1):
            image_draw.line([stroke[0][i], 
                             stroke[1][i],
                             stroke[0][i + 1], 
                             stroke[1][i + 1]],
                            fill=0, width=5)
    image = image.resize((IMG_SIZE, IMG_SIZE))
    return np.array(image, dtype=np.uint8)

  
def vector_output(csvs):
  outputs = {}
  for i, name in enumerate(csvs):
    vec = np.zeros(340)
    vec[i] = 1
    outputs[name[:-4]] = vec
  return outputs

  
def images_and_labels_generator(batch_size):
  outputs = vector_output(os.listdir("train_data"))
  for batch in raw_batch_generator(batch_size):
    batch_images = []
    batch_labels = []
    for e in batch:
      batch_images.append(draw_it(e[0]))
      batch_labels.append(outputs[e[1]])
    batch_images = np.stack(batch_images, axis=0)
    yield batch_images, batch_labels  

<generator object images_and_labels_generator at 0x7f57ad244ca8>

In [0]:
! pip3 install keras

NUM_CLASSES = len(class_labels)
class_to_idx = {c: idx for idx, c in enumerate(class_labels)}

def train_iterator(batch_size):
    for batch in images_and_labels_generator(batch_size):
      print(batch)
        #images = batch[0].astype('float32')
        ### YOUR CODE HERE: normalize images!
        #images = np.expand_dims(images, -1)
        #labels = keras.utils.to_categorical(map(class_to_idx.get, batch[1]), NUM_CLASSES)
        #yield images, labels
        
        
#sample = train_iterator(32).next()
train_iterator(32)

In [0]:
# import necessary building blocks
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, GlobalAveragePooling2D, \
    BatchNormalization



def make_model():
    """
    Define your model architecture here.
    Returns `Sequential` model.
    """
    model = Sequential()

    ### YOUR CODE HERE: replace with a better model! Batch normalization really helps!
    
    model.add(Conv2D(16, (3, 3), padding='same', activation='elu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
    model.add(Conv2D(16, (3, 3), padding='same', activation='elu'))
    model.add(GlobalAveragePooling2D())

    model.add(Dense(64, activation='elu'))
    model.add(Dense(NUM_CLASSES, activation="softmax"))
    
    return model